In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# ---------- PATHS ----------
enhanced_path   = '../Retrieval/Data_Combined/combined_matches_players_all_1_enhanced.csv'

X_full_path     = '../Retrieval/Data_Combined/dota_X_final_full.csv'
X_noleak_path   = '../Retrieval/Data_Combined/dota_X_final_noleak.csv'
X_scaled_path   = '../Retrieval/Data_Combined/dota_X_final_noleak_scaled.csv'
y_path          = '../Retrieval/Data_Combined/dota_y_final.csv'
X_mid_path      = '../Retrieval/Data_Combined/dota_X_midgame.csv'
X_mid_scaled_path = '../Retrieval/Data_Combined/dota_X_midgame_scaled.csv'

# ---------- MID-GAME FILTER CONFIG ----------
MIDGAME_EXCLUDE_KEYS = [
    "tower_damage",
    "hero_damage",
    "roshan",
]

# ---------- LOAD ENHANCED DATA ----------
df = pd.read_csv(enhanced_path)
print("Loaded enhanced dataset:", df.shape)

# ---------- TARGET ----------
target_col = 'radiant_win'
y = df[target_col].astype(int)

# ---------- DROP NON-FEATURE COLUMNS ----------
drop_cols = [
    target_col
]

X_raw = df.drop(columns=drop_cols)
print("After dropping label column:", X_raw.shape)

# ---------- HANDLE CATEGORICALS ----------
cat_cols = X_raw.select_dtypes(include=['object']).columns.tolist()
print("Categorical columns:", cat_cols)

X = pd.get_dummies(X_raw, columns=cat_cols, drop_first=True)
print("After one-hot encoding:", X.shape)

# ---------- HANDLE MISSING VALUES ----------
X = X.replace([np.inf, -np.inf], np.nan)
X = X.fillna(0)
print("Any NaNs left in X?", X.isnull().sum().sum())

# ---------- LEAKY vs NO-LEAK (almost nothing leaks now) ----------
# These "leaky" features DO NOT EXIST in REQUIRED_COLS version → removed entirely
leaky_cols = []

X_full = X.copy()
X_noleak = X.copy()

print("X_full shape:   ", X_full.shape)
print("X_noleak shape: ", X_noleak.shape)

# ---------- SAVE RAW FEATURES & LABELS ----------
X_full.to_csv(X_full_path, index=False)
X_noleak.to_csv(X_noleak_path, index=False)
y.to_csv(y_path, index=False)

print(f"Saved X_full   to {X_full_path}")
print(f"Saved X_noleak to {X_noleak_path}")
print(f"Saved y        to {y_path}")

# ---------- BUILD MID-GAME FEATURE SET ----------
midgame_cols = [
    c for c in X_noleak.columns
    if not any(key in c.lower() for key in MIDGAME_EXCLUDE_KEYS)
]

X_mid = X_noleak[midgame_cols]

print("X_mid shape (mid-game subset):", X_mid.shape)

X_mid.to_csv(X_mid_path, index=False)
print(f"Saved X_mid to {X_mid_path}")

# ---------- SCALED VERSIONS ----------
scaler_noleak = StandardScaler()
X_noleak_scaled = pd.DataFrame(
    scaler_noleak.fit_transform(X_noleak),
    columns=X_noleak.columns
)
X_noleak_scaled.to_csv(X_scaled_path, index=False)
print(f"Saved scaled no-leak X to {X_scaled_path}")

scaler_mid = StandardScaler()
X_mid_scaled = pd.DataFrame(
    scaler_mid.fit_transform(X_mid),
    columns=X_mid.columns
)
X_mid_scaled.to_csv(X_mid_scaled_path, index=False)
print(f"Saved scaled mid-game X to {X_mid_scaled_path}")


Loaded enhanced dataset: (1042190, 50)
After dropping label column: (1042190, 49)
Categorical columns: []
After one-hot encoding: (1042190, 49)
Any NaNs left in X? 0
X_full shape:    (1042190, 49)
X_noleak shape:  (1042190, 49)
Saved X_full   to ../Retrieval/Data_Combined/dota_X_final_full.csv
Saved X_noleak to ../Retrieval/Data_Combined/dota_X_final_noleak.csv
Saved y        to ../Retrieval/Data_Combined/dota_y_final.csv
X_mid shape (mid-game subset): (1042190, 45)
Saved X_mid to ../Retrieval/Data_Combined/dota_X_midgame.csv
Saved scaled no-leak X to ../Retrieval/Data_Combined/dota_X_final_noleak_scaled.csv
Saved scaled mid-game X to ../Retrieval/Data_Combined/dota_X_midgame_scaled.csv
